# DAV 5400 Fall 2019 Week 8 Assignment (30 Points)
## Regular Expressions

- Text data is often in need of “cleaning” and preparation before it can be effectively used for analysis purposes. Consider
the following poorly formatted text string containing names and phone numbers of some residents of the town of
Springfield:
- 
"555-1239Moe Szyslak(636) 555-0113Burns, C. Montgomery555 -6542Rev. Timothy Lovejoy555
8904Ned Flanders636-555-3226Simpson, Homer5553642Dr. Julius Hibbert"

- Use your Python regular expression (“regex”) skills to complete the following tasks:


1. Extract the names of each individual from the unformatted text string and store them in a vector of some sort. When
complete, your vector should contain the following entries:
"Moe Szyslak" "Burns, C. Montgomery" "Rev. Timothy Lovejoy"
"Ned Flanders" "Simpson,Homer" "Dr. Julius Hibbert"


In [1]:
# Imorting the regular expression library , Numpy and Pandas
import re
import numpy as np
import pandas as pd
# Storing the text search space
text = "555-1239Moe Szyslak(636) 555-0113Burns, C. Montgomery555 -6542Rev. Timothy Lovejoy555 8904Ned Flanders636-555-3226Simpson, Homer5553642Dr. Julius Hibbert"

##### Regex:
- divided the expression into three parts first name, middle name(optional and a subject to multiplicity), last name with the same logic as the first

##### Regex breakdown
- Part I: name starting with capital then followed by as many letters as necessary [one or more] finally it is possibly followed by a comma or not and at the end there is one or more spaces

- Part II: Is an optional part that has to start by one capital letter and a one or more lower case letters followed by a possible dot then finally 1 or more spaces, P.S there could be multiple middle names.

- Part III: The last part has basically the same logic as the first part

-  Title: follows a very similar logic, you can consider it part 0

In [2]:
# Regular expression for extracting names.
#Parts: Title                    partI              PartII             PartIII
regex= r"(?:[A-Z][a-z]*\.\s*)?" + r"[A-Z]{1}[a-z]+,?\s+[A-Z{1}a-z*\.?\s+]*[A-Z]{1}[a-z]+"
#Recomling it into the name
name_reg= re.compile(regex)

- The following names:
"Moe Szyslak" "Burns, C. Montgomery" "Rev. Timothy Lovejoy" "Ned Flanders" "Simpson,Homer" "Dr. Julius Hibbert"
- will be extracted from the texts and the honorifics such as Dr, rev will be included


In [3]:
# Storing the names in a Numpy array.
names =name_reg.findall(text)
# Printing the values.
print("These are the six names extracted and stored: \n",names)

These are the six names extracted and stored: 
 ['Moe Szyslak', 'Burns, C. Montgomery', 'Rev. Timothy Lovejoy', 'Ned Flanders', 'Simpson, Homer', 'Dr. Julius Hibbert']


2. Using your new vector containing only the names of the six individuals, complete the following tasks:

   * a. Use your regex skills to rearrange the vector so that all elements conform to the standard “firstname
lastname”, preserving any titles (e.g., “Rev.”, “Dr.”, etc) or middle/second names.

   * b. Construct a logical vector indicating whether a character has a title (i.e., Rev. and Dr.).

   * c. Construct a logical vector indicating whether a character has a middle/second name.

#### Plan Attack for 2.a
So, the expressions that are not ordered are the ones that have a ',' so I would target those specifically and order them within
the entire text


In [4]:
# 2.a
# arranging the the names that are not ordered
main_pattern = r"(?:[A-Z][a-z]*\.\s*)?" + r"[A-Z]{1}[a-z]+,?\s+[A-Z{1}a-z*\.?\s+]*[A-Z]{1}[a-z]+"
extraction= re.findall(main_pattern ,text)

# Unarranged names subset pattern
target_pattern="([A-Z]{1}[a-z]+),([A-Z{1}a-z*\.?\s+]*[A-Z]{1}[a-z]+)"

#The regular expression compiled
regular_exp=re.compile(target_pattern)
# re-ordering the first and last name through groupings in a list comprehnsion.
Question2a=[regular_exp.sub(r'\2, \1', item) for item in extraction]
#here is the re-ordered list
print("This is the ordered version:")
np.array(Question2a)

This is the ordered version:


array(['Moe Szyslak', ' C. Montgomery, Burns', 'Rev. Timothy Lovejoy',
       'Ned Flanders', ' Homer, Simpson', 'Dr. Julius Hibbert'],
      dtype='<U21')

#### Plan of attack for 2.b
- we Construct a logical vector indicating whether a person has an honorific title (i.e., Rev. and Dr.)


In [5]:
# Casting to a series to preform boealon check
data = pd.Series(Question2a)
# Creating the vector
logical_vector = data.str.contains(r'Dr.|dr.|rev.|Rev.|Mr.|Miss.|Excellency.|Mrs.|Sir.|Dame.|sir.|dame.')
print(logical_vector,"\nOr you can look at it this way too: ",list(logical_vector.astype('int64')))

0    False
1    False
2     True
3    False
4    False
5     True
dtype: bool 
Or you can look at it this way too:  [0, 0, 1, 0, 0, 1]


Plan of Attack 2.c
- Construct a logical vector indicating whether a character has a middle/second name by simply checking 
if there are three or more names in a name without an honorific title
or
if there are four or more names in a name with an honorific title
- Meaning we need to create a function that would facilitate this


In [6]:
def wordcount(value):
    '''
    Counts the number of words
    Value is a text expression
    returns an int
    '''
    # Find all non-whitespace patterns.
    list = re.findall("(\S+)", value)
    # Return length of resulting list.
    return len(list)

# demonstration of the word count, Looks great, eh!
print("How Many words?",wordcount(Question2a[0]),"\nHow Many words?",wordcount(Question2a[1]))

How Many words? 2 
How Many words? 3


In [7]:
logical=list(logical_vector.astype('int64'))

middle_vector=[]
for val,lv in zip(Question2a,logical):
    wc= wordcount(val)
    if lv == 0:
        if wc >= 3:
            middle_vector.append(1)
        else:
            middle_vector.append(0)
        
    elif lv ==1:
        if wc >=4:
            middle_vector.append(1)
        else:
            middle_vector.append(0)
print("The second one is the only one that has a middle name",middle_vector)
middle_vector = pd.Series(middle_vector)
middle_vector

The second one is the only one that has a middle name [0, 1, 0, 0, 0, 0]


0    0
1    1
2    0
3    0
4    0
5    0
dtype: int64

## Here is a summary of Question 2 for all a b & c in DataFrame

In [8]:
# initialize list of lists 
data = {
        'Name':list(Question2a)
        ,'Honorific_title_vector':list(logical_vector.astype('int64'))
        ,'middlename_vector':list(middle_vector)
       } 
# Create the pandas DataFrame 
df = pd.DataFrame(data) 
  
# print dataframe. 
df 

,Name,Honorific_title_vector,middlename_vector
0,Moe Szyslak,0,0
1,"C. Montgomery, Burns",0,1
2,Rev. Timothy Lovejoy,1,0
3,Ned Flanders,0,0
4,"Homer, Simpson",0,0
5,Dr. Julius Hibbert,1,0


### Question 3

For question 3 Consider the HTML string We would like to extract the first HTML tag To do so we write the regular expression “<. + >”. Explain why this fails and correct the expression.
This regex is incorrect because it includes the entire expression when we only need a part of it so it would be better if we 
create the following regex and extract the first instance of it


In [9]:
q3="<title>+++BREAKING NEWS+++<title>"
re.findall("<.+>",q3)

['<title>+++BREAKING NEWS+++<title>']

r"\<\w+\>" would be alot more useful to get the first instance of the title tag

In [10]:
# 3. Consider the HTML string <title>+++BREAKING NEWS+++<title>. We would like to extract the first HTML tag (i.e.,
# “<title>”). To do so we write the regular expression “<.+>”. Explain why this fails and correct the expression.
test="<!DOCTYUPE>howurhgouwrhgwrhgirhgpwrih<title>+++BREAKING NEWS+++<title> hjsthoirhipwrhpwjrhpijrohjwrp"
# Creating the regex to extract it
title_regex=r"\<\w+\>"
entire= re.findall(title_regex ,test)[0]
print(entire)

<title>


4. Consider the string “(5-3) ^2=5^2-2*5*3+3^2” conforms to the binomial theorem. We would like to extract the
formula in the string. To do so we write the regular expression “[^0-9=+*()] +”. Explain why this fails and correct the
expression.


As you can see below it doesn't get the required values it only gets the  certain operators and not the formula, that’s why it is not working because it lack digits and operators


In [11]:
test2 ="(5-3)^2=5^2-2*5*3+3^2"
rg=r"[^0-9=+*()]+"
fl= re.findall(rg ,test2)
print(fl)

['-', '^', '^', '-', '^']


This is to get the entire equation (5-3)^2=5^2-2*5*3+3^2 we get it like this

In [12]:

test3 ="(5-3)^2=5^2-2*5*3+3^2"
rg=r"\(\d[-|+]\d\)\^\d=\S+\b"
final= re.findall(rg ,test2)[0]
print(final)


(5-3)^2=5^2-2*5*3+3^2


# Thank You : Omar M. H